### RDD 내부 동작 원리
1. 특정 컬럼의 고유한 값의 개수를 세기
2. A로 시작하는 것들 찾기
3. 스크린에 결과 출력하기 

위 과정을 코드로 나눠보면  
첫 번째 .map(lambda v:(v, 1))를 이용 스파크가 A를 포함하는 단어를 모음 
.filter(lambda val: bal.startwith('A')) 를 사용해서 A로 시작하는 단어를 필터링 
.reduceByKey(operator.add)를 호출   
트랜스폼   

.collect()함수를 호출. 
액션

In [14]:
import findspark, pyspark 
findspark.find()

'C:\\bigdata\\spark-2.4.5-bin-hadoop2.7'

In [13]:
sc.stop()

In [15]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession 
conf = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [16]:
data = sc.parallelize(
[
    ('Amber',22), ('Alfred',23), ('Skye',4), ('Albert',12), ('Amber',9)
]
)

In [17]:
data_from_file = sc.textFile('C:/Users/admin/Desktop/spark_data/VS14MORT.txt')

In [18]:
type(data_from_file)

pyspark.rdd.RDD

.textFile()
첫 번째 : 데이터 파일의 위치(경로)
두 번째 : 파티션의 개수(노드 당? 클러스터 당? 두 개 ~ 4개 정도의 파티션으로 관리) 

지원되는 파일 시스템 :  
Windows ( NTFS, FAT )   
Mac (HFS+)   
분산파일시스템 (HDFS, S3, Cassandra)  

경로 : 특수문자 '['__']' 포함 할 수 없다 

지원되는 텍스트 포맷 : CSV, JSON, HIVE 테이블, JDBC 드라이버, parquet, 압축파일 (Gzip) tar.gz, gz, zip 

리스트, 튜플, 딕셔너리를 동시에 구성할 수 있다.

In [19]:
data_heterogenous = sc.parallelize([
    ('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]
]).collect()

In [20]:
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

In [21]:
data_heterogenous[1]['Porsche']

100000

In [22]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

In [23]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

In [24]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row : row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

스파크는 두가지 모드로 동작 할 수 있다 (로컬모드, 클러스트 모드)   
스파크가 로컬모드로 동작할 때는 파이썬을 실행시키는 것과 다르지 않다.  
(바뀐 것은 대부분 구문상의 것들과 데이터의 코드가 분리된 워커 프로세스 사이에서 복사 될 수 있다는 약간의 구조적인 차이)   
UDF 사용시 특별한 주의 없이 같은 코드를 클러스터 모드에서 실행하면 골치 아픈 일들이 많이 생긴다.  

# Transformations
  
.map()  
  
map() 함수는 가장 많이 사용하는 함수, 이 함수는 RDD의 각 엘리먼트에 적용이 된다.  



In [25]:
data_2014 = data_from_file_conv.map(lambda row : int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

.filter() 

이 함수를 이용해서 데이터셋으로부터 특정 조건에 맞는 엘리먼트를 선택할 수 있다 

In [26]:
data_filtered = data_from_file_conv.filter(lambda row : row[5] == 'F' and row[21] == '0' )

In [27]:
data_filtered.count()

6

In [28]:
data_filteredtered1 = data_from_file_conv.filter(lambda row : row[16] == '2014' and row[21] == '0')

In [29]:
data_filteredtered1.count()

22

.flatMap()  
.flatMap() 함수는 .map() 함수와 비슷하게 동작. 리스트가 아닌 평면화된 결과를 리턴한다. 

In [30]:
data_2014_flat = data_from_file_conv.flatMap(lambda row : (row[16], int(row[16]) + 1))

In [31]:
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

.distinct()

이 함수는 특정 컬럼에서의 중복된 값을 제거해 고유한 값을 리스트로 리턴한다. 

In [32]:
distinct_gender = data_from_file_conv.map(lambda row : row[5]).distinct().collect()

In [33]:
distinct_gender

['-99', 'M', 'F']

이 함수는 많은 자원을 사용하는 함수이다, 데이터를 섞는 연산을 포함하기 때문에 정말 필요한 경우에만 사용해야 한다.  

.sample()  

.sample() 함수는 데이터셋으로부터 임의로 추출된 샘플을 리턴한다.  
첫 번째 : 중복 허용 여부   
두 번째 : 리턴할 데이터셋과 전체 데이터셋 간의 크기 비율   
세 번째 : 임의의 숫자를 생성하기 위한 시드값  

In [34]:
fraction = 0.1 
data_sample = data_from_file_conv.sample(False, fraction, 555) ### 555 - 랜덤시드 값 

In [35]:
data_sample.take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '058', ' ', '37', '17', '08',
        '  ', '4', 'D', '3', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I250',
        '214', '062', '   ', '21', '03', '11I250 ', '61I272 ', '62E669 ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '03',
        'I250 ', 'E669 ', 'I272 ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [36]:
print('Original dataset : {0}, sample : {1}'.format(data_from_file_conv.count(), data_sample.count()))

Original dataset : 2631171, sample : 262602


.leftOuterJoin()

왼쪽을 기준으로 두개의 데이터셋을 합침 

In [37]:
rdd1 = sc.parallelize([('a',1), ('b',4), ('c',10)])
rdd2 = sc.parallelize([('a',4), ('a',1), ('b',6), ('d',15)])

In [38]:
rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, 6)), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [39]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1))]

In [40]:
rdd5 = rdd1.intersection(rdd2) # 교집합 
rdd5.collect()

[('a', 1)]

.repartition()  

데이터셋 재파티션하면 데이터가 나눠지는 파티션의 개수가 바뀐다.  
부하가 많이 가는 작업이다.  

In [41]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect()) # glom() : 파티션에 대한 정보를 가지고 옴 

4

### Actions 

액션은 데이터셋에서 스케줄된 테스크를 실행한다.  

.take()  
map() 함수처럼 .take()함수 역시 가장 유용한 함수이다.   
이 함수는 하나의 파티션에서 가장 위에 있는 n행을 리턴한다.  
RDD 전체를 리턴하는 .collect()보다 자주 사용함.   
큰 데이터셋일수록 중요하다.  

In [42]:
data_first = data_from_file_conv.take(1)
data_first

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

데이터로부터 임의의 샘플을 얻고 싶다면 .takeSample() 함수를 사용.  
이 함수는 세 개의 파라미터  
첫 번째 : 중복 허용 여부   
두 번째 : 리턴되는 데이터 갯수   
세 번째 : 랜덤 시스값   

In [43]:
data_take_sample = data_from_file_conv.takeSample(False, 1, 557)
data_take_sample

[array(['2', '  ', '4', '1', '11', 'M', '1', '046', ' ', '35', '15', '07',
        '  ', '1', 'D', '7', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'G258',
        '194', '111', '   ', '37', '01', '11G258 ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'G258 ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '02', ' ',
        ' ', '3', '2', '100', '7'], dtype='<U40')]

.reduce()

.reduce() 함수는 특정 함수를 사용해 RDD의 개수를 줄인다. 

In [44]:
rdd1.map(lambda row : row[1]).reduce(lambda x, y : x + y)

15

rdd1의 값 리스트를 map() 트랜스포메이션을 이용해 생성했고 그 결과를 처리하기 위해 .reduce() 함수를 이용.    
.reduce() 함수는 각각의 파티션에서 합계 함수를 수행하고 마지막 집계가 수행되는 드라이버 노드에 그 합계를 리턴 

**주의**   
리듀서로 전달되는 함수는 결합 법칙이 성립해야 한다.   
(엘리먼트의 순서가 바뀌어도 결과에는 영향을 주지 않아야 한다.)  
교환 법칙이 성립해야 한다.  
(피연산자의 순서가 바뀌어도 결과는 같아야 한다.)  

결합 법칙의 예 : ( 5 + 2 ) + 3 = 5 + ( 2 + 3 ) 
교환 법칙의 예 : 5 + 2 + 3 = 3 + 2 + 5

어떤 함수를 리듀서로 정할지 신중하게 결정해야 한다. 

In [45]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 1)

데이터를 현재 결과와 다음 결과를 나누는 방법으로 리듀스하면 예상되는 결과는 10이 나온다. (파티션이 하나일 때) 

In [46]:
works = data_reduce.reduce(lambda x, y : x/y)
works

10.0

파티션을 세개로 나누고, 결과 확인

In [47]:
data_reduce1 = sc.parallelize([1, 2, .5, .1, 5, .2], 3)

In [48]:
data_reduce1.reduce(lambda x, y : x/y)

0.004

키 값을 기반으로 리듀스 하기 

In [49]:
data_key = sc.parallelize([('a', 4), ('b', 3), ('c', 2), ('a', 8), ('d', 2), ('b', 1), ('d', 3)], 4)
data_key.reduceByKey(lambda x, y : x + y).collect()

[('b', 4), ('c', 2), ('a', 12), ('d', 5)]

.count()  
.count() 함수는 전체 데이터셋을 드라이버로 옮기지 않는다. 

In [50]:
data_reduce.count()

6

In [51]:
# 이렇게 하면 드라이버로 들어오니 주의
# len(data_reduce.collect()) 

데이터셋이 키 - 밸류 형태로 있을 경우, 고유한 키의 수를 구하기 위해 countByKey() 함수를 사용한다.

In [52]:
data_key.countByKey().items()

dict_items([('a', 2), ('b', 2), ('c', 1), ('d', 2)])

.saveAsTextFile()

각 파티션을 분리된 파일에 저장

In [55]:
data_key.saveAsTextFile('./data/PySpark_data_key.txt')

모든 행이 스트링으로 인식, 뒤쪽부터 읽고 싶으면 뒤쪽부터 파싱 해야 함 

In [71]:
def parseInpu(row) :
    import re
    
    pattern = re.compile(r'\(\'([a-z])\',([0-9])\)')
    row_split = pattern.split(row)
    return (row_split[1], int(row_split[2]))

In [72]:
data_key_reread = sc.textFile('./data/PySpark_data_key.txt').map(parseInpu)

In [73]:
data_key_reread.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 42.0 failed 1 times, most recent failure: Lost task 0.0 in stage 42.0 (TID 393, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\bigdata\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\bigdata\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\bigdata\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\bigdata\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-71-5f7e779aec34>", line 6, in parseInpu
IndexError: list index out of range

	at